## 라이브러리

In [1]:
import os

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm

import re
import pickle

from konlpy.tag import Mecab

In [2]:
base_path = '/home/hyejeongeun/chatbot/Data/QnA/'
save_path = '/home/hyejeongeun/chatbot/Data/QnA/'

## 데이터 로드

In [3]:
df = pd.read_csv(base_path+'qna_table_hanspell.csv')

In [4]:
df[:1]

,product_id,product_name,product_option,user_id,user_buyer,is_secret,label,question,question_time,answer,answer_time,question_clean,answer_clean,question_spellcheck,answer_spellcheck
0,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 우드,3077236,True,False,환불,상부 연결 목재 중 하나가 나사산이 덜 파져 있네요;; 조립하다 목재가 갈라져버립니...,2021-03-18T16:52:34.000+09:00,안녕하세요 고객님 먼데이하우스입니다.\r\n불편을 드려 죄송합니다.\r\n유선상 연...,2021-03-18T18:19:46.000+09:00,상부 연결 목재 중 하나가 나사산이 덜 파져 있네요 조립하다 목재가 갈라져버립니다 ...,안녕하세요 고객님 먼데이하우스입니다 불편을 드려 죄송합니다 유선상 연락을 드렸으나 ...,<None>,안녕하세요 고객님 먼데이 하우스입니다 불편을 드려 죄송합니다 유선상 연락을 드렸으나...


In [5]:
df.shape

(95190, 15)

In [6]:
# hanspell 이후 <None> 제거
df = df[(df['question_spellcheck']!='<None>')&(df['answer_spellcheck']!='<None>')]
df.reset_index(drop=True, inplace=True)

In [7]:
df.shape

(94999, 15)

## Mecab

In [8]:
mecab = Mecab()

In [9]:
def make_mecab_tokens(comment):

    # mecab 돌리기
    comment_mecab = []
    for i in range(len(comment)):
        if comment[i] == 'NaN':
            comment_mecab.append('nan')
        else:
            comment_mecab.append(mecab.pos(comment[i]))

    # 조사 제거
    morpheme = ['NNG','NNP','NNB','NNBC','NR','NP','VV','VA','VX','VCP','VCN','MM','MAG','MAJ','IC']    
    tmp = []
    comment_tokens = []
    
    for tok in comment_mecab:
        for t in tok:
            if t[1] in morpheme:
                tmp.append(t[0])
        if len(tmp)==0:
            comment_tokens.append('')
        else:
            comment_tokens.append(tmp)
        tmp = []

    # 한 문장으로 결합
    comment_tokens_str = [' '.join(re) for re in comment_tokens]

    return comment_tokens_str

In [10]:
# question
question = df['question_spellcheck'].astype(str)

df['question_mecab'] = make_mecab_tokens(question)

In [11]:
# answer
answer = df['answer_spellcheck'].astype(str)

df['answer_mecab'] = make_mecab_tokens(answer)

In [12]:
df = df[(df['question_mecab']!='')&(df['answer_mecab']!='')] # NaN 제거
df.reset_index(drop=True, inplace=True)

In [13]:
df.shape

(94742, 17)

In [29]:
df[:1]

,product_id,product_name,product_option,user_id,user_buyer,is_secret,label,question,question_time,answer,answer_time,question_clean,answer_clean,question_spellcheck,answer_spellcheck,question_mecab,answer_mecab
0,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 우드,3076525,True,False,상품,배송 언제 되는걸까요...?,2021-03-18T15:31:16.000+09:00,안녕하세요 고객님~~먼데이하우스입니다.\r\n유선상 연락 및 안내드렸습니다.\r\n...,2021-03-18T15:52:19.000+09:00,배송 언제 되는걸까요,안녕하세요 고객님 먼데이하우스입니다 유선상 연락 및 안내드렸습니다 감사합니다 1,배송 언제 되는 걸까요,안녕하세요 고객님 먼데이 하우스입니다 유선상 연락 및 안내드렸습니다 감사합니다 1,배송 언제 되,안녕 고객 먼데이 하우스 유선 연락 및 안 감사


## QNA[SEP]ANS

In [15]:
df['qna_hanspell_sep'] = df['question_spellcheck'].values+' [SEP] '+df['answer_spellcheck'].values + '\t'
df['qna_mecab_sep'] = df['question_mecab'].values+' [SEP] '+df['answer_mecab'].values + '\t'

In [16]:
df['qna_hanspell_sep']

0        배송 언제 되는 걸까요 [SEP] 안녕하세요 고객님 먼데이 하우스입니다 유선상 연락...
1        구매하였는데 배송 메모는 무시해주세요 다른 곳에서 산거 수정을 못했어요 빠른 배송 ...
2        조립하다가 육각 나사 렌치 돌리는 부분이 다 갈려서 조립을 못해요 육각 나사 렌치 ...
3        주문해서 다음 주 월요일 3 22에 받고 싶은데 언제 주문하면 될까요 [SEP] 안...
4        나사 너무 안 들어가는데 원래 이렇게 나사가 안 들어가나요 힘을 줘도 절대 끝까지 ...
                               ...                        
94737    배송비는 따로 청구되나요 경기도 파주 야당동입니다 [SEP] 안녕하세요 고객님 노루...
94738    변기 교체하려는데요 기존 변기 수거도 해주시고 기사님이 배송하시고 시공 설치해주시는...
94739    1 이거 결제하면 더 이상의 비용은 없나요 예를 들면 철거비나 쓰레기 처리 비용 같...
94740    상품만 사면 시공도 다 무료인가요 가게 변기 교체 좀 하려고 하는데요 지역은 인천 ...
94741    상품 가격만 나와있어서요 설치기사님 비용은 어떻게 되나요 지역은 경기도 기흥입니다 ...
Name: qna_hanspell_sep, Length: 94742, dtype: object

In [17]:
df['qna_mecab_sep']

0               배송 언제 되 [SEP] 안녕 고객 먼데이 하우스 유선 연락 및 안 감사\t
1        구매 배송 메모 무시 주 다른 곳 산거 수정 못 배송 부탁 [SEP] 안녕 고객 먼...
2        조립 육각 나사 렌치 돌리 부분 다 조립 못 육각 나사 렌치 주 수 있 [SEP] ...
3        주문 다음 주 월요일 받 싶 언제 주문 [SEP] 안녕 고객 먼데이 하우스 월요일 ...
4        나사 너무 안 들어가 원래 이렇게 나사 안 들어가 힘 절대 끝 안 [SEP] 안녕 ...
                               ...                        
94737    배송 비 따로 청구 경기도 파주 야당동 [SEP] 안녕 고객 노루 페인트 홈 인테리...
94738    변기 교체 요 기존 변기 수거 주 기사 배송 시공 설치 주 것 맞 지역 서울 서초구...
94739    이거 결제 더 이상 비용 없 예 들 철 거비 쓰레기 처리 비용 같 경기도 파주시 문...
94740    상품 사 시공 다 무료 가 변기 교체 좀 하 하 지역 인천 청천동 변기 좌변기 면기...
94741    상품 가격 있 설치 기사 비용 어떻게 되 지역 경기도 기흥 [SEP] 설치 비용 포합\t
Name: qna_mecab_sep, Length: 94742, dtype: object

## Labeling

In [18]:
# label 수치형으로 변환
cat = pd.Series(df['label'])
labels, uniques = pd.factorize(cat)
labels, uniques

(array([0, 1, 0, ..., 0, 0, 0]),
 Index(['상품', '배송', '교환', '환불', '기타', '반품'], dtype='object'))

In [19]:
df['num_label'] = labels
df['num_label'] = df['num_label'].astype(str) # str으로 변환

In [20]:
df[['label','num_label']].value_counts()

label  num_label
상품     0            44904
배송     1            30148
교환     2             8357
환불     3             5780
기타     4             3049
반품     5             2483
dtype: int64

In [21]:
df.loc[df['num_label'] =='4', 'num_label'] = 'tmp'
df.loc[df['num_label'] =='5', 'num_label'] = '4'
df.loc[df['num_label'] =='tmp', 'num_label'] = '5'

In [22]:
df[['label','num_label']].value_counts()

label  num_label
상품     0            44904
배송     1            30148
교환     2             8357
환불     3             5780
기타     5             3049
반품     4             2483
dtype: int64

In [23]:
df.shape

(94742, 20)

In [32]:
# num_label -1 제거
df = df[df['num_label']!='-1']

In [33]:
df.shape

(94721, 20)

In [34]:
# 기타 미포함
df = df[df['num_label']!='5']

In [35]:
df.shape

(91672, 20)

In [37]:
# csv 저장 후 내보내기
df.to_csv(save_path+'qna_table_mecab.csv', index=False)

In [38]:
df.reset_index(drop=True, inplace=True)

In [39]:
df.shape

(91672, 20)

## KoBERT용 데이터

- mecab + 카테고리 5개(기타 미포함)

In [40]:
# 4. mecab + 카테고리 5개(기타 미포함)

# list 생성
qna = list(df['qna_mecab_sep'].values + df['num_label'].astype(str).values + '\n')

# train test split
qna_train, qna_test = train_test_split(qna, test_size=0.2, shuffle=True, random_state=218)

# txt 내보내기
with open(save_path+'qna_mecab_drop_train.txt', 'w') as f:
    f.writelines(qna_train)
    
with open(save_path+'qna_mecab_drop_test.txt', 'w') as f:
    f.writelines(qna_test)

In [43]:
# 4. mecab + 카테고리 5개(기타 미포함)

train_4 = pd.read_csv(save_path+'qna_mecab_drop_train.txt', header=None, sep='\t')
test_4 = pd.read_csv(save_path+'qna_mecab_drop_test.txt', header=None, sep='\t')

print('shape')
print(train_4.shape, test_4.shape)
print('——————————————')

# length
train_4_len = np.array(train_4[0].apply(lambda x:len(x.split(' '))))
test_4_len = np.array(test_4[0].apply(lambda x:len(x.split(' '))))

# print max_len
print('max_len')
print(np.max(train_4_len), np.max(test_4_len))
print('——————————————')

# print train label values_counts
print('train value_counts')
print(train_4[1].value_counts())
print('——————————————')
print('test value_counts')
print(test_4[1].value_counts())

shape
(73337, 2) (18335, 2)
——————————————
max_len
208 183
——————————————
train value_counts
0    35838
1    24169
2     6716
3     4659
4     1955
Name: 1, dtype: int64
——————————————
test value_counts
0    9066
1    5979
2    1641
3    1121
4     528
Name: 1, dtype: int64
